In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


In [2]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

# 三个自定义函数
def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


# prompt中有两个input_variable
prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        # 获取到用户输入的foo，将其输入给length_function，最后赋给a
        "a": itemgetter("foo") | RunnableLambda(length_function),
        # 同时会去到用户输入的foo和bar，输入给multiple_length_function，赋给b
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    # a和b输入给prompt
    | prompt
    | model
)

In [3]:
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 = 12')

## <font color=red>TODO</font> 2. RunnableConfig

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig

import json


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05
